In [1]:
from mphpc import HPCApp
import os

# Setting the client_id of the hpc app and pass the access_token to using the app through SDK
hpc = HPCApp(client_id="5fd66c68-50e9-474a-b55d-148777ae3efd",
                    access_token=os.environ.get('MP_ACCESS_TOKEN'))

In [2]:
# The heartbeat capability to show the state of server and the HPC resource
print(hpc.status())

{"description":"System ready","message":"user is available.","status":"available","system":"daint"}



## initialize a new job

using capability `new_trasformation`, the app create a folder under the user repository path in the remote HPC resource. 
The remote path name is returned for futher files transfor operations and as `transformation_id` for job submission.

In [3]:
# The resource_id returned will be the remote path
resource_id = hpc.new_job(config=None)
print(resource_id)

8d16388c-7354-427a-9931-8c1555028a89


In [4]:
!ssh daint.cscs.ch "cd /scratch/snx3000/jyu/firecrest/jusong_yu/ && ls -l"

total 48
drwxr-xr-x 2 jyu mr0 4096 Apr  8 09:50 023eeb90-241b-4dac-81af-88aa4973923e
drwxr-xr-x 2 jyu mr0 4096 Apr  5 13:00 0e7d0608-d3d2-4ea7-b778-47de81e21931
drwxr-xr-x 2 jyu mr0 4096 Apr  8 03:56 20e93a11-8fc4-42c0-b2b3-e8061880b7b4
drwxr-xr-x 2 jyu mr0 4096 Apr  5 13:41 4dab5be5-65d5-4303-baf0-5f1722b27c53
drwxr-xr-x 2 jyu mr0 4096 Apr  8 03:25 54d0af78-be1a-4da1-ac9f-ac9599f7b4a2
drwxr-xr-x 2 jyu mr0 4096 Apr  8 03:48 6f96f400-b157-448a-b478-27aa2a5ee522
drwxr-xr-x 2 jyu mr0 4096 Apr 28 13:43 8d16388c-7354-427a-9931-8c1555028a89
drwxr-xr-x 2 jyu mr0 4096 Apr  8 03:51 b47105b6-b765-4b4c-8ac9-2a0e4d3a3bcd
drwxr-xr-x 2 jyu mr0 4096 Apr  5 14:17 ba96c061-f706-47eb-9acd-78666fbb3ca1
drwxr-xr-x 2 jyu mr0 4096 Apr  8 10:42 e990fde0-960e-49a3-b5af-7abf3b115ffd
drwxr-xr-x 2 jyu mr0 4096 Apr 27 15:40 f1701e07-7e48-4b53-a915-6e0a9caad4cf
drwxr-xr-x 2 jyu mr0 4096 Apr 28 12:57 f6ed9397-0bf2-4db7-975f-61b6df4af447


# Upload input files and slurm job script to the path

The capabilities `updateDataset`, `getDataset` and `deleteDataset` are for upload, download and delete files on remote path.

However, instead of remote path only it is also require to pass filename to manipulate the files which is not completely compatible with the initial designs of these capabilities which are for database in the first place. 
I hack it a bit to pass the filename by a json object.

In [5]:
cmd = f'ls -l /scratch/snx3000/jyu/firecrest/jusong_yu/{resource_id}'
!ssh daint.cscs.ch $cmd

total 0


In [6]:
! head -n 10 ./sample_files/colloid.in

# Big colloid particles and small LJ particles

units		lj
atom_style	sphere
dimension	2

lattice		sq 0.01
region		box block 0 30 0 30 -0.5 0.5
create_box	2 box
create_atoms	1 box


In [7]:
# send LAMMPS input file and job script to remote folder
hpc.upload(resource_id=resource_id, source_path='./sample_files/colloid.in')

hpc.upload(resource_id=resource_id, source_path='./sample_files/submit.sh')

Not 200 but 201, please check
Not 200 but 201, please check


In [8]:
cmd = f'ls -l /scratch/snx3000/jyu/firecrest/jusong_yu/{resource_id}'
!ssh daint.cscs.ch $cmd

total 8
-rw-r--r-- 1 jyu mr0 1066 Apr 28 13:44 colloid.in
-rw-r--r-- 1 jyu mr0  592 Apr 28 13:44 submit.sh


In [9]:
# download the file
content = hpc.download(resource_id=resource_id, filename='submit.sh')
print(content)

#!/bin/bash -l
#SBATCH --job-name="lammps-colloid"
#SBATCH --account="mrcloud"
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jusong.yu@epfl.ch
#SBATCH --time=00:10:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-core=1
#SBATCH --ntasks-per-node=4
#SBATCH --cpus-per-task=1
#SBATCH --partition=debug
#SBATCH --constraint=mc
#SBATCH --hint=nomultithread

module load daint-mc/20.08
module load LAMMPS/03Mar20-CrayGNU-20.08

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

'srun' '-n' '4' '/apps/daint/UES/jenkins/7.0.UP02/mc/easybuild/software/LAMMPS/03Mar20-CrayGNU-20.08/bin/lmp_mpi' '-in' 'colloid.in'



## Submit job and check the jobs list

`start_transformation` corresponding to capability `startTransformation` which will do actuall job submit into the scheduler system and return the job info.
Then I can using `get_transformation_list` to retrieve all the jobs of current user in remote resource.


In [10]:
import json

jblst = hpc.list_job()
print(json.dumps(jblst, indent=2))

{
  "37534830": {
    "jobid": "37534830",
    "name": "job_name",
    "nodelist": "nid01197",
    "nodes": "1",
    "partition": "debug",
    "start_time": "2022-04-05T13:00:14",
    "state": "COMPLETED",
    "time": "00:19:12",
    "time_left": "2022-04-05T13:00:30"
  },
  "37582275": {
    "jobid": "37582275",
    "name": "lammps-colloid",
    "nodelist": "nid01197",
    "nodes": "1",
    "partition": "debug",
    "start_time": "2022-04-08T03:25:14",
    "state": "COMPLETED",
    "time": "00:40:48",
    "time_left": "2022-04-08T03:25:48"
  },
  "37587906": {
    "jobid": "37587906",
    "name": "lammps-colloid",
    "nodelist": "nid00011",
    "nodes": "1",
    "partition": "debug",
    "start_time": "2022-04-08T10:41:28",
    "state": "COMPLETED",
    "time": "01:22:48",
    "time_left": "2022-04-08T10:42:37"
  },
  "38015471": {
    "jobid": "38015471",
    "name": "lammps-colloid",
    "nodelist": "nid00010",
    "nodes": "1",
    "partition": "debug",
    "start_time": "2022-04-

In [11]:
rsp = hpc.run_job(resource_id=resource_id)
print(rsp)

{"job_data_err":"","job_data_out":"","job_file":"/scratch/snx3000/jyu/firecrest/jusong_yu/8d16388c-7354-427a-9931-8c1555028a89/submit.sh","job_file_err":"/scratch/snx3000/jyu/firecrest/jusong_yu/8d16388c-7354-427a-9931-8c1555028a89/slurm-38038203.out","job_file_out":"/scratch/snx3000/jyu/firecrest/jusong_yu/8d16388c-7354-427a-9931-8c1555028a89/slurm-38038203.out","jobid":38038203,"result":"Job submitted","userid":"624c1d1104ba3511f2b24240"}



In [12]:
jblst = hpc.list_job()
print(jblst)

{'37534830': {'jobid': '37534830', 'name': 'job_name', 'nodelist': 'nid01197', 'nodes': '1', 'partition': 'debug', 'start_time': '2022-04-05T13:00:14', 'state': 'COMPLETED', 'time': '00:19:12', 'time_left': '2022-04-05T13:00:30'}, '37582275': {'jobid': '37582275', 'name': 'lammps-colloid', 'nodelist': 'nid01197', 'nodes': '1', 'partition': 'debug', 'start_time': '2022-04-08T03:25:14', 'state': 'COMPLETED', 'time': '00:40:48', 'time_left': '2022-04-08T03:25:48'}, '37587906': {'jobid': '37587906', 'name': 'lammps-colloid', 'nodelist': 'nid00011', 'nodes': '1', 'partition': 'debug', 'start_time': '2022-04-08T10:41:28', 'state': 'COMPLETED', 'time': '01:22:48', 'time_left': '2022-04-08T10:42:37'}, '38015471': {'jobid': '38015471', 'name': 'lammps-colloid', 'nodelist': 'nid00010', 'nodes': '1', 'partition': 'debug', 'start_time': '2022-04-27T15:40:22', 'state': 'COMPLETED', 'time': '00:14:24', 'time_left': '2022-04-27T15:40:34'}, '38038203': {'jobid': '38038203', 'name': 'lammps-colloid', '

In [ ]:
# resource_id = 'f1701e07-7e48-4b53-a915-6e0a9caad4cf'
cmd = f"cat /scratch/snx3000/jyu/firecrest/jusong_yu/{resource_id}/log.lammps"
!ssh daint.cscs.ch $cmd

In [ ]:
# download the file
content = hpc.download(resource_id=resource_id, filename='log.lammps')
print(content)